In [ ]:
import numpy as np
import numpy as np
import pandas as  pd
import tensorflow as tf
from tensorflow import keras
#import keras
import matplotlib.pyplot as plt 
from tensorflow.keras import layers
from sklearn import *
import math
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import scipy
tf.autograph.set_verbosity(0)
from sklearn import linear_model as lm  # Used for solving linear regression problems
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import cmath
import math

In [ ]:
def reIm2complex_sb(sbi):
    
  # It takes a matrix of 1 subband but Re and Im part concatenated. It merges 
  # them together into a complex subband of MPCs
  
  # Inputs
  # sbi: matrix [tti x p·2]
  
  # Outputs
  # sbfold: matrix [tti x p]
  

    ncol_out = int(sbi.shape[1]/2)
    nrow_out = int(sbi.shape[0])

    sbfold = np.empty([(nrow_out), (ncol_out)], dtype = 'complex_')
    sbi = sbi.values
    for i in range((ncol_out)):
        
        sbfold[:, i] = np.array([r * (math.cos(phi) + math.sin(phi)*1j) for r, phi in zip(sbi[:, i], sbi[:, i+ncol_out])])
        
    return sbfold

In [ ]:
#fold_matrix <- function(unfold_mat){
#  # It takes an unfold matrix of MPCs and returns a forded one
#  
#  # Inputs
#  # sbi: matrix [tti x p·2·sb]
#  
#  # Outputs
#  # sbfold: matrix [p  x sb x tti]
#  
#  sb = 4
#  sbcolreim <- ncol(unfold_mat)/sb
#  nrows_sb <- nrow(unfold_mat)
#  
#  p <- sbcolreim/2
#  
#  sb1 <- unfold_mat[, 1:sbcolreim]
#  sb2 <- unfold_mat[, (sbcolreim+ 1) : (2*sbcolreim)]
#  sb3 <- unfold_mat[, (2*sbcolreim + 1) : (3*sbcolreim)]
#  sb4 <- unfold_mat[, (3*sbcolreim + 1) : ncol(unfold_mat)]
#  
#  sb1_fold <- reIm2complex_sb(sb1) 
#  sb2_fold <- reIm2complex_sb(sb2) 
#  sb3_fold <- reIm2complex_sb(sb3) 
#  sb4_fold <- reIm2complex_sb(sb4) 
#  
#  fold_mat <-  sb1_fold
#  fold_mat <- abind(fold_mat, sb2_fold, along = 3)
#  fold_mat <- abind(fold_mat, sb3_fold, along = 3)
#  fold_mat <- abind(fold_mat, sb4_fold, along = 3)
#  
#  aperm(fold_mat, c(2,3,1))
#  
#}

In [ ]:
def predict_ar_1step(theta, y_target):
    """Predicts the value y_t for t = p+1, ..., n, for an AR(p) model, based on the data in y_target using
    one-step-ahead prediction.

    :param theta: array (p,), AR coefficients, theta=(a1,a2,...,ap).
    :param y_target: array (n,), the data points used to compute the predictions.
    :return y_pred: array (n-p,), the one-step predictions (\hat y_{p+1}, ...., \hat y_n) 
    """

    n = len(y_target)
    p = theta.shape[1]
    
    # Number of steps in prediction
    m = n-p
    y_pred = np.zeros(m+1)
    for i in range(m):
        y_pred[i] =  (np.flip(y_target[i:i+p]) * theta).sum()
        
    return y_pred

In [ ]:
def fit_ar(y, p):
    """Fits an AR(p) model. The loss function is the sum of squared errors from t=p+1 to t=n.

    :param y: array (n,), training data points
    :param p: int, AR model order
    :return theta: array (p,), learnt AR coefficients
    """

    # Number of training data points
    n = y.shape[0]
    
    # Construct the regression matrix
    Phi = np.zeros((n-p, p))# <COMPLETE THIS LINE>
    for j in range(p):
        Phi[:,j] = y[(p-(j+1)): (n-(j+1)), 0] # <COMPLETE THIS LINE>
    
    # Drop the first p values from the target vector y
    yy = y[p:]  # yy = (y_{t+p+1}, ..., y_n)

    # Here we use fit_intercept=False since we do not want to include an intercept term in the AR model
    regr = lm.LinearRegression(fit_intercept=False)
    regr.fit(Phi,yy)    

    return regr.coef_

In [ ]:
def reconstruct_validation(mpc, y_pred, X_test):
    
    valid_reconstruct = np.empty((y_pred.shape[0], mpc*8,))

    indexes = np.arange(mpc * 8)

    idx_sb1 = np.arange(0,(mpc*2))
    idx_sb2 = np.arange((mpc*2),(mpc*4))
   

    idx_sb1_dis = np.array(list(set(indexes) - set(idx_sb1)))
    idx_sb2_dis = np.array(list(set(indexes) - set(idx_sb2)))
   
    sb_counter_1 = 0
    sb_counter_2 = -1
   
    for i in range(y_pred.shape[0],):

        if sb_counter_1%4==0:

            valid_reconstruct[i, idx_sb1] = X_test[i,:]
            valid_reconstruct[i, idx_sb1_dis] = y_pred[i,:]

        elif sb_counter_2%4==0:

            valid_reconstruct[i, idx_sb2] = X_test[i,:]
            valid_reconstruct[i, idx_sb2_dis] = y_pred[i,:]

        sb_counter_1 += 1
        sb_counter_2 += 1
        
    return valid_reconstruct


In [ ]:


def acf(x, lags=None):
    """ Computes the empirical autocorralation function.
    :param x: array (n,), sequence of data points
    :param lags: int, maximum lag to compute the ACF for. If None, this is set to n-1. Default is None.
    :return gamma: array (lags,), values of the ACF at lags 0 to lags
    """

    gamma = np.correlate(x, x, mode='full')  # Size here is always 2*len(x)-1
    gamma = gamma[int((gamma.size - 1) / 2):]  # Keep only second half
    if lags is not None and lags < len(gamma):
        gamma = gamma[0:lags + 1]
    return gamma / gamma[0]

In [ ]:

    
def acfplot(x, lags=None, conf=0.95):
    """Plots the empirical autocorralation function.
    :param x: array (n,), sequence of data points
    :param lags: int, maximum lag to compute the ACF for. If None, this is set to n-1. Default is None.
    :param conf: float, number in the interval [0,1] which specifies the confidence level (based on a central limit
                 theorem under a white noise assumption) for two dashed lines drawn in the plot. Default is 0.95.
    :return:
    """

    n = len(x)
    y = acf(x, lags)
    lags = len(y)
    
    lag_vec = np.arange(lags)
    
    c = norm.isf((1-conf)/2,loc=0,scale=1/np.sqrt(n)) # Use inverse survival function (=1-cdf) at half the confidence interval
    plt.plot(lag_vec,c*np.ones(lags),'k--',linewidth=1, label=f"{100*conf}% confidence")
    plt.plot(lag_vec,-c*np.ones(lags),'k--',linewidth=1)
    
    plt.stem(lag_vec, y, linefmt='-', markerfmt=' ', basefmt="k ", use_line_collection=True, label="Empirical ACF")
    plt.plot(lag_vec, 0*lag_vec, 'k-')
    plt.title(f"Empirical ACF")
    plt.legend() 

In [ ]:
def nrmse(y_pred, y_test):
    
    num = tf.math.reduce_sum(tf.square(y_pred - y_test))
    num = num / y_test.shape[1]
    
    den = tf.math.reduce_sum(tf.square(y_test))/y_test.shape[1]
    
    return (num/den).numpy()

def mse(y_pred, y_test):
    
    num = tf.math.reduce_sum(tf.square(y_pred - y_test))
    num = num / y_test.shape[1]
    
    return num.numpy()

def mae(y_pred, y_test):
    
    num = tf.math.reduce_sum(tf.abs(y_pred - y_test))
    num = num / y_test.shape[1]
    
    return num.numpy()


In [ ]:
def plot_train_test(history, name_ds):
    
    plt_1 = plt.figure(figsize=(6, 3))

    plt.plot(history.history['MSE'])
    #plt.plot(history.history['val_total_loss'])
    plt.ylabel('Loss', fontsize ='xx-large')
    plt.xlabel('epoch', fontsize ='xx-large')
    plt.plot(history.history['val_MSE_val'])

    plt.legend(['train', 'test'], loc='upper center', fontsize ='12')
    plt.title('Model Loss During Training {}'.format(name_ds), fontsize='xx-large')

    plt.show()

In [ ]:
def  plot_pred_vs_true(y_pred, y_test, name_ds):
    
    plt_1 = plt.figure(figsize=(6, 3))
    plt.scatter(y_test.flatten(), y_pred.flatten(), s=2)
    plt.title('VED Predictions {}'.format(name_ds), fontsize='xx-large')
    plt.ylabel('Prediction', fontsize ='xx-large')
    plt.xlabel('Background Truth', fontsize ='xx-large')

    plt.show()

In [ ]:
def plot_sorted_vs_true(y_pred, y_test, name_ds):

    plt_1 = plt.figure(figsize=(10, 6))

    plt.plot(np.sort(y_test.flatten()))
    x_idx = np.arange(len(y_test.flatten()))
    plt.scatter(x_idx, y_pred.flatten()[np.argsort(y_test.flatten())], s=2, c="red")

    plt.title('Sorted Predictions of Validation Set {}'.format(name_ds), fontsize='xx-large')
    plt.ylabel('Prediction', fontsize ='xx-large')
    plt.xlabel('Background Truth', fontsize ='xx-large')
    plt.legend(['Sorted Background Truth', 'Predictions'], loc='upper right', fontsize ='xx-large')

    plt.show()


In [ ]:
def plot_1TTI(y_test, y_pred, X_test, samp_num, name_ds): 

    plt_1 = plt.figure(figsize=(10, 6))
    sample_1 = np.r_[ y_test[samp_num,], X_test[samp_num,]]
    idx = np.arange(len(sample_1))
    plt.scatter(np.arange(len(sample_1)), sample_1, s=2, c="black")
    plt.scatter(idx[-len(y_pred[samp_num,]):], y_pred[samp_num,], color='red',  s=2)
    plt.title('Prediction of TTI n={} {}'.format(samp_num, name_ds) , fontsize ='xx-large')
    plt.legend(['Background Truth', 'Predictions'],labelcolor = ['black', 'red'] ,loc='upper right', fontsize ='xx-large')
    plt.ylabel('Feature', fontsize ='xx-large')
    plt.xlabel('Amplitude', fontsize ='xx-large')

    plt.show()
    
    

In [ ]:
def plot_MPC_overtime(data, mpc, name_ds, Real):
    
    if Real:
        plt.plot(data[:,0], label='1st MPC SB1')
        plt.plot(data[:, mpc*2], label='1st MPC SB2')
        plt.plot(data[:, mpc*4], label='1st MPC SB3')
        plt.plot(data[:, mpc*6], label='1st MPC SB4')


        plt.title('plot of 1st MPC in 4 subbands Over Time (Real Part) {}'.format(name_ds),  fontsize='xx-large')
        plt.xlabel('TTI', fontsize='xx-large')
        plt.ylabel('Amplitude', fontsize='xx-large')
        plt.legend(fontsize='xx-large')
        plt.tight_layout()
        plt.show() 
    
    else:
        plt.plot(data[:,mpc], label='1st MPC SB1')
        plt.plot(data[:,mpc*3], label='1st MPC SB2')
        plt.plot(data[:,mpc*5], label='1st MPC SB3')
        plt.plot(data[:,mpc*7], label='1st MPC SB4')


        plt.title('plot of 1st MPC in 4 subbands Over Time (Imaginary Part) {}'.format(name_ds),  fontsize='xx-large')
        plt.xlabel('TTI', fontsize='xx-large')
        plt.ylabel('Amplitude', fontsize='xx-large')
        plt.legend(fontsize='xx-large')
        plt.tight_layout()
        plt.show() 

In [ ]:
def NMSE(true_chan, pred_chan):
    
    chann_diff = true_chan - pred_chan
    
    num = (np.sqrt(sum(sum(((chann_diff.conj().T).dot(chann_diff))**2)))).real
    den = (np.sqrt(sum(sum(((true_chan.conj().T).dot(true_chan))**2)))).real
    
    return num/den
 